# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import gmaps
import os


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = "../output_data/cities.csv"

weather_df = pd.read_csv(file)
weather_df.head()


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:

# Configure gmaps
gmaps.configure(api_key = "gkey")

# Use the lat and lng columns as the locations variable
locations = weather_df[["Lat", "Lng"]]

# Use the humidity column of the df as the weight variable
humidity = weather_df["Humidity"]

In [4]:

#Add Heatmap layer to map.

fig = gmaps.figure(center=(25.0, 15.0), zoom_level = 2.0)

# Generate a new layer on the map - Heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity,
                                 dissipating = False, max_intensity = 100,
                                 point_radius = 3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
Temp = weather_df["Max Temp"]
Temp

0      37.50
1      78.80
2      22.20
3      44.60
4      55.99
       ...  
543    80.60
544    84.20
545    61.00
546    49.74
547    55.86
Name: Max Temp, Length: 544, dtype: float64

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [15]:

# dropping NA values
weather_df.dropna(inplace = True)

#  cities Temp 80 > and < 70
# Switching to 85 for the high to improve results
perfect_temperature = weather_df.loc[(Temp < 85) & (Temp > 70)]
perfect_temperature

# Next narrow it down further by looking for lower wind speeds (< 10)
less_wind = perfect_temperature.loc[(perfect_temperature["Wind Speed"] < 10)]
less_wind

# remove all clouds in the sky by setting cloudiness equal to zero

ideal_weather = less_wind.loc[(less_wind["Cloudiness"] <= 15)]
ideal_weather

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
20,20,teknaf,1,BD,1558378758,85,20.86,92.31,84.66,8.23
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
87,87,ambilobe,2,MG,1558378770,92,-13.19,49.05,74.76,4.52
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
125,125,arraial do cabo,0,BR,1558378491,78,-22.97,-42.02,84.20,4.70
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
221,221,marawi,12,PH,1558378796,93,8.00,124.29,77.00,5.01
268,268,dingle,4,PH,1558378836,80,11.00,122.67,79.08,2.73


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [19]:
hotel_df = ideal_weather
hotel_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
20,20,teknaf,1,BD,1558378758,85,20.86,92.31,84.66,8.23
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
87,87,ambilobe,2,MG,1558378770,92,-13.19,49.05,74.76,4.52
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
125,125,arraial do cabo,0,BR,1558378491,78,-22.97,-42.02,84.20,4.70


In [21]:
#creating a new column and calling it hotel name
hotel_df["Hotel Name"] = ""
hotel_df.head()

C:\Users\16517\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
20,20,teknaf,1,BD,1558378758,85,20.86,92.31,84.66,8.23,
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,
87,87,ambilobe,2,MG,1558378770,92,-13.19,49.05,74.76,4.52,
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,
125,125,arraial do cabo,0,BR,1558378491,78,-22.97,-42.02,84.20,4.70,


In [23]:

# Prepare the api requests by setting up the url and parameters using the "nearby search" type to perform this request
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Parameters for search based 
params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [25]:
# Run API request to find the hotels within 5,000 meters of the cities in our dataframe
counter = 0

# Use for loop to run through each row in the dataframe
for index, perfect_city in hotel_df.iterrows():
    
    # Increase counter as we move through to the each row in the dataframe
    counter += 1
    
    # Pull the city name, lat, and lng values from the row in hotel_df
    lat = perfect_city["Lat"]
    lng = perfect_city["Lng"]
    city_name = perfect_city["City"]
    
    # add keyword to params dict
    params["location"] = (f"{lat}, {lng}")

    # assemble url and make API request
    #print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(url, params)
    # Convert to json
    response_json = response.json()
    
    # Extract the results and store them in a well-named variable
    results = response_json["results"]
    
    # Print statement to show where we are in the counting process of the dataframe search
    print(f"{counter} of {len(hotel_df)}: Looking for hotels near {city_name}")
    #print(json.dumps(results, indent = 4, sort_keys = True))   
    
    # If possible save the hotel name to dataframe
    # Need to use try logic hear so that even if it doesn't find a value for one city, it can move on and find the others
    try:
        hotel_name = results[0]["name"]
        print(f"Hotel found! {hotel_name}")
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]

    # Display a message if unable to find a hotel with the given parameters
    except (KeyError, IndexError):
        print(f"Sorry, we could not find a hotel near {city_name}")
        hotel_df.loc[index, "Hotel Name"] = "NA"
    
    # Adding in a one second sleep interval between queries to avoid exceeding the API query limits
    time.sleep(1)
    
# Print end of search once searching is completed
print("-----------------------------")
print(f"Finished collecting hotel information from {len(hotel_df)} cities with ideal weather conditions")

hotel_df

1 of 19: Looking for hotels near teknaf
Sorry, we could not find a hotel near teknaf


C:\Users\16517\.conda\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


2 of 19: Looking for hotels near ponta do sol
Sorry, we could not find a hotel near ponta do sol
3 of 19: Looking for hotels near ambilobe
Sorry, we could not find a hotel near ambilobe
4 of 19: Looking for hotels near cidreira
Sorry, we could not find a hotel near cidreira
5 of 19: Looking for hotels near arraial do cabo
Sorry, we could not find a hotel near arraial do cabo
6 of 19: Looking for hotels near morondava
Sorry, we could not find a hotel near morondava
7 of 19: Looking for hotels near vaini
Sorry, we could not find a hotel near vaini
8 of 19: Looking for hotels near nador
Sorry, we could not find a hotel near nador
9 of 19: Looking for hotels near marawi
Sorry, we could not find a hotel near marawi
10 of 19: Looking for hotels near dingle
Sorry, we could not find a hotel near dingle
11 of 19: Looking for hotels near mogok
Sorry, we could not find a hotel near mogok
12 of 19: Looking for hotels near gunjur
Sorry, we could not find a hotel near gunjur
13 of 19: Looking for ho

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
20,20,teknaf,1,BD,1558378758,85,20.86,92.31,84.66,8.23,NA
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,NA
87,87,ambilobe,2,MG,1558378770,92,-13.19,49.05,74.76,4.52,NA
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,NA
125,125,arraial do cabo,0,BR,1558378491,78,-22.97,-42.02,84.20,4.70,NA
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,NA
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,NA
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,NA
221,221,marawi,12,PH,1558378796,93,8.00,124.29,77.00,5.01,NA
268,268,dingle,4,PH,1558378836,80,11.00,122.67,79.08,2.73,NA


In [27]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [29]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))